## **Visualizing the spread of COVID-19 in the state of Florida**
### By Matthew Keeran
### 3/12/2020

In [1]:
import imageio
import numpy as np
import pandas as pd
import plotly.figure_factory as ff
from datetime import datetime, timedelta
%matplotlib inline

#### **Read FIPS mappings for FL counties**

In [2]:
"""
    State & County FIPS codes (US Census)
    https://www.census.gov/geographies/reference-files/2016/demo/popest/2016-fips.html

code used to preprocess the data from csv above ^ 
I extracted only the 67 counties from FL from the entire list
fips = pd.read_csv('COVID-19/data/county_mappings.csv')
def decounty_county(s):
    s = str(s)
    return s[:-7].lower()

def format_county_code(code):
    code = str(code)
    if len(code) == 1:
        return '00' + code
    elif len(code) == 2:
        return '0' + code
    else:
        return code
    
fips['County'] = fips['Area Name (including legal/statistical area description)'].apply(lambda x: decounty_county(x))
fips['State Code (FIPS)'] = fips['State Code (FIPS)'].apply(lambda x: str(x))
fips['County Code (FIPS)'] = fips['County Code (FIPS)'].apply(lambda x: format_county_code(x))
fips['FIPS'] = fips['State Code (FIPS)'] + fips['County Code (FIPS)']
fips.drop([
        'Summary Level',
        'State Code (FIPS)',
        'County Code (FIPS)',
        'County Subdivision Code (FIPS)',
        'Place Code (FIPS)',
        'Consolidtated City Code (FIPS)',
        'Area Name (including legal/statistical area description)'
    ], axis=1, inplace=True)
fips.to_csv('COVID-19/data/FL_FIPS.csv', index=False)
"""
fips = pd.read_csv('data/FL_FIPS.csv')
fips.head(10)

,County,FIPS
0,alachua,12001
1,baker,12003
2,bay,12005
3,bradford,12007
4,brevard,12009
5,broward,12011
6,calhoun,12013
7,charlotte,12015
8,citrus,12017
9,clay,12019


#### **Read data**

In [3]:
df = pd.read_csv('data/FL_COVID-19.csv')
df.drop(['dead','death_date','time_publication_to_death'], axis=1, inplace=True)
df['published'] = df['published'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
print(len(df))
df.head()

42


,published,county,age,sex,travel_related
0,2020-03-01,manatee,63,male,no
1,2020-03-01,hillsborough,29,female,yes
2,2020-03-05,santa rosa,71,male,yes
3,2020-03-06,broward,75,male,no
4,2020-03-06,broward,65,male,no


#### **resolve FIPS code for each county**

In [4]:
def fipify(x):
    # probably a better way to do this?
    return fips[fips['County'] == x]['FIPS'].iloc[0]

df['FIPS'] = df['county'].apply(lambda x: fipify(x))
df.head()

,published,county,age,sex,travel_related,FIPS
0,2020-03-01,manatee,63,male,no,12081
1,2020-03-01,hillsborough,29,female,yes,12057
2,2020-03-05,santa rosa,71,male,yes,12113
3,2020-03-06,broward,75,male,no,12011
4,2020-03-06,broward,65,male,no,12011


In [6]:
dates = list(df['published'].unique())
dates

[numpy.datetime64('2020-03-01T00:00:00.000000000'),
 numpy.datetime64('2020-03-05T00:00:00.000000000'),
 numpy.datetime64('2020-03-06T00:00:00.000000000'),
 numpy.datetime64('2020-03-07T00:00:00.000000000'),
 numpy.datetime64('2020-03-08T00:00:00.000000000'),
 numpy.datetime64('2020-03-09T00:00:00.000000000'),
 numpy.datetime64('2020-03-10T00:00:00.000000000'),
 numpy.datetime64('2020-03-11T00:00:00.000000000'),
 numpy.datetime64('2020-03-12T00:00:00.000000000'),
 numpy.datetime64('2020-03-13T00:00:00.000000000')]

In [7]:
# date_freqs = [{date1},{date2}]
# date_freqs[0] = {"date": date1, "counties": {}}
# date_freqs[0]['counties'] = {'county1': freq1, 'county2': freq2}
dates_freqs = []
# for every date get all cases up to and including that day
for date in dates:
    counties = {}
    tmp = df[df['published'] <= date]
    # then get all the frequencies for each county
    for county in list(tmp['county'].unique()):
        tmp2 = tmp[tmp['county'] == county]
        counties[tmp2['FIPS'].iloc[0]] = len(tmp2)
    dates_freqs.append({"date" : date, "counties" : counties})

In [8]:
fip_list = fips['FIPS'].tolist()

#### **Create Chloropleth (sounds like an evil alien species, and is about as much fun to work with)**
#### I wish I could provide a documentation page which shows all the methods and attributes of the object, but alas I could not find one. 
#### best I could do https://plot.ly/python/county-choropleth/, 
#### yes I know its deprecated but the new way's (https://plot.ly/python/choropleth-maps/) tutorial doesn't show how to do individual states at the county level.

In [9]:
maxi      = max(list(dates_freqs[-1]['counties'].values()))
endpts    = [0,1,2,3,4,5,10]#,15,20,30,40,50,75,100]
fips      = fip_list
filenames = []
for date in dates_freqs:
    counties = date['counties']
    values   = [counties.get(fip, 0) for fip in fip_list]

    colorscale = ["#008712","#c8ff70","#f4fc00", "#bda000", "#fc9700", "#a34502", "#fc6900", "#fc5400", "#fc0000", "#fc0000", "#fc0000"]
    
    fig = ff.create_choropleth(
        fips=fips, values=values, scope=['Florida'], show_state_data=True,
        colorscale=colorscale, 
        binning_endpoints=endpts, 
        round_legend_values=True,
        plot_bgcolor='rgb(0,0,255)',
        paper_bgcolor='rgb(0,0,255)',
        legend_title='Cases by County',
        title='Distribution of COVID-19 in FL: {}'.format(date['date'].astype(str)[:10]),
        county_outline={'color': 'rgb(255,255,255)', 'width': 0.5},
        font={"color":"#945600", "size":17}
    )
    fig.layout.template = None
    file_nm = 'images/{}_fl_covid-19.png'.format(date['date'].astype(str)[:10])
    fig.write_image(file_nm)
    filenames.append(file_nm)

#### **Make Gif from daily images**

In [10]:
dt = datetime.utcnow().date().isoformat()
images = []
for file in filenames:
    images.append(imageio.imread(file))
imageio.mimsave('{}_FL.gif'.format(dt), images)